In [1]:
import os

In [2]:
%pwd

'd:\\vehicle_insurance_fraud_detection\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'd:\\vehicle_insurance_fraud_detection'

In [5]:
from dataclasses import dataclass
from pathlib import Path 

@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    X_train_data_path: Path
    y_train_data_path: Path
    X_test_data_path: Path
    y_test_data_path: Path
    model_name: str

In [6]:
from vifd.constants import *
from vifd.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath=CONFIG_FILE_PATH,
            schema_filepath=SCHEMA_FILE_PATH
    ):
        self.config = read_yaml(config_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer

        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir=config.root_dir,
            X_train_data_path=config.X_train_data_path,
            y_train_data_path=config.y_train_data_path,
            X_test_data_path=config.X_test_data_path,
            y_test_data_path=config.y_test_data_path,
            model_name=config.model_name
        )

        return model_trainer_config
    

In [8]:
import pandas as pd
import os
from vifd import logger
from sklearn.linear_model import LogisticRegression
import joblib

[2024-01-19 22:56:20,250: INFO: utils: Note: NumExpr detected 12 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.]
[2024-01-19 22:56:20,252: INFO: utils: NumExpr defaulting to 8 threads.]


In [9]:
class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config

    def train(self):
        X_train = pd.read_csv(self.config.X_train_data_path)
        y_train = pd.read_csv(self.config.y_train_data_path)['fraud_reported_Y'].values
        X_test = pd.read_csv(self.config.X_test_data_path)
        y_test = pd.read_csv(self.config.y_test_data_path)['fraud_reported_Y'].values

        log_reg = LogisticRegression(random_state=42)
        log_reg.fit(X_train, y_train)

        joblib.dump(log_reg, os.path.join(self.config.root_dir, self.config.model_name))


In [10]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer_config = ModelTrainer(config=model_trainer_config)
    model_trainer_config.train()

except Exception as e:
    raise e

[2024-01-19 22:56:22,774: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-01-19 22:56:22,779: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-01-19 22:56:22,781: INFO: common: created directory at: artifacts]
[2024-01-19 22:56:22,782: INFO: common: created directory at: artifacts/model_trainer]
